In [1]:
from pyspark.sql import SparkSession

DIRPATH = "C:/Users/sanjay s risbud/Downloads/sundog/"
spark = SparkSession.builder.master("local").appName("Exercise").getOrCreate()

In [18]:
from pyspark.sql.functions import split, explode, array
lines = spark.read.text(DIRPATH + "marvel-graph.txt")
lines = lines.withColumn("parsed", split(lines["value"], " ", limit=2))
lines = lines.withColumn("hero", lines["parsed"][0]).withColumn("appeared_with", explode(split(lines["parsed"][1], " ")))
df = lines.select(["hero", "appeared_with"])
df.show()

+----+-------------+
|hero|appeared_with|
+----+-------------+
|5988|          748|
|5988|         1722|
|5988|         3752|
|5988|         4655|
|5988|         5743|
|5988|         1872|
|5988|         3413|
|5988|         5527|
|5988|         6368|
|5988|         6085|
|5988|         4319|
|5988|         4728|
|5988|         1636|
|5988|         2397|
|5988|         3364|
|5988|         4001|
|5988|         1614|
|5988|         1819|
|5988|         1585|
|5988|          732|
+----+-------------+
only showing top 20 rows



In [30]:
from pyspark.sql.functions import max, min, lit
grouped_df = df.groupBy("hero").count()
stats = grouped_df.select(max("count"), min("count")).first()
stats

Row(max(count)=1937, min(count)=1)

In [39]:
most_df = grouped_df.filter(grouped_df["count"] == stats[0]).withColumn("remarks", lit("Most Connected"))
least_df = grouped_df.filter(grouped_df["count"] == stats[1]).withColumn("remarks", lit("Least Connected"))
stats_df = most_df.union(least_df)
stats_df.show()

+----+-----+---------------+
|hero|count|        remarks|
+----+-----+---------------+
| 859| 1937| Most Connected|
| 467|    1|Least Connected|
| 577|    1|Least Connected|
|3490|    1|Least Connected|
|3489|    1|Least Connected|
|2139|    1|Least Connected|
|1089|    1|Least Connected|
|1841|    1|Least Connected|
|4517|    1|Least Connected|
|5028|    1|Least Connected|
| 835|    1|Least Connected|
|1408|    1|Least Connected|
|4784|    1|Least Connected|
|4945|    1|Least Connected|
|4602|    1|Least Connected|
|6411|    1|Least Connected|
|3014|    1|Least Connected|
|3298|    1|Least Connected|
|2911|    1|Least Connected|
|2117|    1|Least Connected|
+----+-----+---------------+



In [41]:
from pyspark.sql.types import StructType, IntegerType, StringType
schema  = StructType().add("hero", IntegerType(), nullable=True).add("name", StringType(), nullable=True)
names_df = spark.read.csv(DIRPATH + "marvel-names.txt", sep=" ", schema=schema)
final_df = stats_df.join(names_df, on="hero")
final_df.show()

+----+-----+---------------+--------------------+
|hero|count|        remarks|                name|
+----+-----+---------------+--------------------+
| 859| 1937| Most Connected|     CAPTAIN AMERICA|
| 467|    1|Least Connected|        BERSERKER II|
| 577|    1|Least Connected|              BLARE/|
|3490|    1|Least Connected|MARVEL BOY II/MARTIN|
|3489|    1|Least Connected|MARVEL BOY/MARTIN BU|
|2139|    1|Least Connected|      GIURESCU, RADU|
|1089|    1|Least Connected|       CLUMSY FOULUP|
|1841|    1|Least Connected|              FENRIS|
|4517|    1|Least Connected|              RANDAK|
|5028|    1|Least Connected|           SHARKSKIN|
| 835|    1|Least Connected|     CALLAHAN, DANNY|
|1408|    1|Least Connected|         DEATHCHARGE|
|4784|    1|Least Connected|                RUNE|
|4945|    1|Least Connected|         SEA LEOPARD|
|4602|    1|Least Connected|         RED WOLF II|
|6411|    1|Least Connected|              ZANTOR|
|3014|    1|Least Connected|JOHNSON, LYNDON BAIN|


In [42]:
spark.stop()